In [15]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pickle
from mutation import *
from crossover import *
from rmp import *
from mfea import *
from task import *
from llm import *
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# load_dotenv()

# GPT_API_KEY = os.getenv("GPT_API_KEY")

# llm = GPTModel(GPT_API_KEY, "gpt-3.5-turbo-0125", 0.7)

In [17]:
cec2017_tasks = get_10_tasks()

In [21]:
num_indis_per_task = 100
indi_len = 50
mutation = PolynomialMutation(5, 0.02)
crossover = SBXCrossover(mutation, eta=2)
# crossover = BLXCrossover()
rmp = AdaptiveRMPMatrix(rmp_pop_size=5, num_gen=5, pc=0.8, pm=0.1)

np.random.seed(0)
amfea = AMFEA(num_indis_per_task, indi_len, cec2017_tasks, crossover, mutation, rmp)

Initialization:
Task 0:
Best Fitness: 125473.96699307997
Mean Fitness: 166392.121167927

Task 1:
Best Fitness: 319204.0909081603
Mean Fitness: 503987.30720511527

Task 2:
Best Fitness: 346653.31463481847
Mean Fitness: 486737.256905658

Task 3:
Best Fitness: 41.0604113392313
Mean Fitness: 49.714217403069476

Task 4:
Best Fitness: 4058761661.357497
Mean Fitness: 6371677088.876898

Task 5:
Best Fitness: 21.482750101043788
Mean Fitness: 21.715487176250566

Task 6:
Best Fitness: 86.24595513767974
Mean Fitness: 99.8559653446189

Task 7:
Best Fitness: 18243.689210172182
Mean Fitness: 21153.768897608556

Task 8:
Best Fitness: 74.8950497645563
Mean Fitness: 122.64795239117853

Task 9:
Best Fitness: 88658.75615256254
Mean Fitness: 124389.07711105635



In [22]:
bfs, mfs = amfea.fit(num_gen=1000, monitor=True, monitor_rate=100, llm_rate=100)

Creating strategy
Creating strategy
Creating strategy
Creating strategy
Creating strategy
Evaluating strategy
Strategy: 
Calculate the Euclidean distance between the mean vectors of each pair of tasks. Adjust RMP values inversely proportional to the distance, assigning higher probabilities to more similar tasks.
Analyze the variance vectors of each task to identify tasks with similar spread or diversity. Increase RMP values for task pairs with similar variance, as individuals in these tasks are likely to be more compatible.
Consider the correlation between mean vectors and variance vectors of tasks. Adjust RMP values based on both the distance between mean vectors and the similarity in variance, prioritizing task pairs with both similarities.
Utilize clustering techniques on the population statistics to group tasks with similar characteristics. Set RMP values higher within clusters and lower between clusters, promoting mating among tasks with shared features.
Creating code...
RMP funct

<string>:14: RuntimeWarning: overflow encountered in exp


Evaluating strategy
Strategy: 
Calculate the Mahalanobis distance between the mean vectors of tasks using the population variance as the covariance matrix. Decrease RMP values for task pairs with higher Mahalanobis distance, discouraging mating between tasks that are statistically more dissimilar.
Perform a Principal Component Analysis (PCA) on the population mean vectors to identify the main directions of variance across tasks. Adjust RMP values based on the principal components to prioritize mating between tasks that exhibit similar dominant directions in the search space.
Compute the Jensen-Shannon Divergence between the probability distributions of individuals within each pair of tasks. Adjust RMP values based on the divergence score, assigning lower probabilities to tasks with more distinct distribution patterns.
Utilize a Gaussian Mixture Model (GMM) to model the distribution of individuals within each task. Adjust RMP values based on the GMM parameters to favor mating between ta

C:\Users\madmo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\decomposition\_pca.py:586: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
C:\Users\madmo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\decomposition\_pca.py:586: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
C:\Users\madmo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\decomposition\_pca.py:586: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
C:\Users\madmo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\decomposition\_pca.py:586: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
C:\Users\madmo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\decomposition\_pca.py:586: RuntimeWarning: invalid value encountered in divide
  explain

Performance: -30547.71251932952
Reverse...
Evaluating strategy
Strategy: 
Calculate the Mahalanobis distance between the mean vectors of tasks considering the variance-covariance matrices. Decrease RMP values for task pairs with higher Mahalanobis distance to discourage mating between tasks with unique distribution patterns.
Perform a t-test on the population means of task pairs to assess the statistical significance of differences. Keep RMP values constant regardless of the t-test results, avoiding favoring mating between tasks with significantly different mean vectors.
Use the Jensen-Shannon divergence to measure the dissimilarity between the probability distributions of tasks. Decrease RMP values for task pairs with higher Jensen-Shannon divergence to discourage mating between tasks with distinct distribution profiles.
Apply a dimensionality reduction technique such as PCA on the population variance matrices to identify latent factors influencing task similarities. Adjust RMP values

...

Performance: -573418958.3604712
Crossover...
Evaluating strategy
Strategy: 
Calculate the cosine similarity between the mean vectors of task pairs. Adjust RMP values to favor mating between tasks with higher cosine similarity, promoting convergence towards similar regions in the search space.
Analyze the Mahalanobis distance between the variance vectors of task pairs. Adjust RMP values inversely proportional to the Mahalanobis distance, favoring mating between tasks with more diverse spread of individuals to encourage exploration of different regions.
Cluster tasks based on the correlation matrix of their mean vectors. Adjust RMP values to favor mating within the same cluster, promoting exploration within groups of tasks that exhibit similar patterns of relationships between dimensions.
Calculate the Jensen-Shannon Divergence between the probability distributions of task pairs. Adjust RMP values based on the Divergence, assigning higher probabilities to task pairs with high Divergence 

<string>:22: RuntimeWarning: divide by zero encountered in scalar divide
<string>:22: RuntimeWarning: invalid value encountered in scalar multiply


Performance: -528061068.9480181
Reverse...
Evaluating strategy
Strategy: 
Calculate the cosine dissimilarity between the mean vectors of task pairs. Adjust RMP values to favor mating between tasks with lower similarity, discouraging convergence towards similar regions in the search space.
Analyze the Euclidean distance between the variance vectors of task pairs. Adjust RMP values directly proportional to the Euclidean distance, assigning higher probabilities to task pairs with higher distance to promote exploration of diverse regions in the search space.
Cluster tasks based on the covariance matrix of their mean vectors. Adjust RMP values to favor mating between tasks from different clusters, promoting exploration across groups of tasks that exhibit different patterns of relationships between dimensions.
Calculate the Kullback-Leibler Divergence between the probability distributions of task pairs. Adjust RMP values inversely proportional to the Divergence, assigning lower probabilities

<string>:35: ComplexWarning: Casting complex values to real discards the imaginary part
<string>:36: ComplexWarning: Casting complex values to real discards the imaginary part
<string>:13: RuntimeWarning: invalid value encountered in sqrt


Evaluating strategy
Strategy: 
Calculate the sine similarity between the mean vectors of task pairs. Adjust RMP values to be inversely proportional to the sine similarity, favoring mating between tasks that have opposite directions in the search space.
Analyze the Euclidean distance between the variance vectors of task pairs using the population covariance matrices. Adjust RMP values based on the Euclidean distance, giving higher probabilities to task pairs with larger Euclidean distances to promote mating between tasks with different spread in a multivariate space.
Perform a Singular Value Decomposition on the mean vectors to identify the dominant directions of variance. Adjust RMP values to favor mating between tasks that lie along the same principal components, promoting exploration along similar dimensions in the search space.
Calculate the Kullback-Leibler Divergence between the probability distributions of task pairs. Adjust RMP values directly proportional to the Divergence, ass

In [22]:
# num_tasks = len(tasks)
# fig, axes = plt.subplots(num_tasks, 2)
# fig.tight_layout()
# for i in range(num_tasks):
#     axes[i][0].plot(bfs[i])
#     axes[i][1].plot(mfs[i])